In [1]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

df = pd.read_csv('data/IMDb_Cleaned_Reviews.csv')
analyzer = SentimentIntensityAnalyzer()

# Define a function to compute the compound sentiment score for a given text
def get_sentiment_score(text):
    scores = analyzer.polarity_scores(text)
    return scores['compound']

# Apply the function to the "Review Content" column and create a new column "Sentiment Score"
df['Sentiment Score'] = df['Review Content'].apply(get_sentiment_score)

# Save the updated DataFrame to a new CSV file
df.to_csv('data/IMDb_Cleaned_Reviews_With_Sentiment.csv', index=False)

# Print the first few rows to verify
print(df.head())

    Movie ID                         Review Title  \
0  tt0435705                             magic do   
1  tt0435705  mining another philip k  dick story   
2  tt0435705                        exciting film   
3  tt0435705                         far expected   
4  tt0435705                    pretty good stuff   

                                      Review Content   Review Date  \
0  nicholas cage  julianne moore  jessica biel st...  Jan 26, 2013   
1  next comes story written philip k  dick  like ...  May 23, 2014   
2  watched film not knowing anything heard  came ...  Oct 12, 2008   
3  next is much better movie expected see  read r...   May 5, 2007   
4  cris is man gift  can see future  least bit in...   May 2, 2007   

   Worldwide BO  Sentiment Score  
0      77621983           0.8958  
1      77621983           0.9893  
2      77621983           0.9246  
3      77621983           0.9960  
4      77621983           0.7557  
